Primero conectamos Python a la base de datos SQL

In [1]:
import mysql.connector
import pandas as pd

In [2]:
# Conexión a la base de datos
db = mysql.connector.connect(
    host="192.168.1.66",
    user="root",
    password="admin",
    database="Stock"
)

In [3]:
cursor = db.cursor()

## QUERYs PARA MANEJAR LA BD

Cargamos los df de cada tabla

In [4]:
import pandas as pd

archivo_excel = pd.ExcelFile("base de datos.xlsx")

df_maestro = pd.read_excel(archivo_excel, 'Maestro UdxBultoprov2023')
df_linea = pd.read_excel(archivo_excel, 'linea')
df_sector = pd.read_excel(archivo_excel, 'sector')
df_proveedor = pd.read_excel(archivo_excel, 'proveedor')
df_familia = pd.read_excel(archivo_excel, 'familia')
df_subfamilia = pd.read_excel(archivo_excel, 'subfamilia') 

In [21]:
import pandas as pd

archivo_excel = pd.ExcelFile("base de datos.xlsx")
df_corregido = pd.read_excel(archivo_excel, "Maestro UdxBultoprov2023")
query = 'SELECT * FROM maestro_articulos'
df_maestro=pd.read_sql(query, db)

C:\Users\mdasilva\AppData\Local\Temp\ipykernel_11116\4009077703.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_maestro=pd.read_sql(query, db)


In [24]:
df_corregido['codart']=df_corregido['codart'].astype(int)
df_corregido['codart']=df_corregido['codart'].astype(str)

In [25]:
df_corregido.rename(columns={'codart': 'Codigo'}, inplace=True)

In [26]:
df_combined = pd.merge(df_maestro, df_corregido, on='Codigo', suffixes=('_actual', '_corregido'))

In [28]:
diferencias = df_combined[df_combined['CodFamilia'] != df_combined['codfamilia']]

In [29]:
diferencias

,Codigo,Descripcion,UxB,Estado,CodLinea,CodSector,CodFamilia,CodSubFamilia,CodProveedor,descart,unidxbult,estado,codlinea,codsector,codfamilia,codsubfamilia,codproveedor
2,100000,ACTIMEL NATURAL PACK X400 GR,1,SUSPEN,400,8,11,3,302,ACTIMEL NATURAL PACK X400 GR,1.0,SUSPEN,400.0,8.0,44,3,302
3,100001,ACTIMEL VAINILLA PACK X 400 GR,1,SUSPEN,400,8,11,3,302,ACTIMEL VAINILLA PACK X 400 GR,1.0,SUSPEN,400.0,8.0,44,3,302
4,100002,ACTIMEL FRUTILLA PACK X400 GR,1,SUSPEN,400,8,11,3,302,ACTIMEL FRUTILLA PACK X400 GR,1.0,SUSPEN,400.0,8.0,44,3,302
5,100003,ACTIMEL SECRETOS PACK x 400 GR,1,SUSPEN,400,8,11,3,302,ACTIMEL SECRETOS PACK x 400 GR,1.0,SUSPEN,400.0,8.0,44,3,302
6,100004,ACTIMEL 0 MULTIFRUTA PACK AHORRO 12x100 GR,1,SUSPEN,400,8,11,3,302,ACTIMEL 0 MULTIFRUTA PACK AHORRO 12x100 GR,1.0,SUSPEN,400.0,8.0,44,3,302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33904,9983,*YBA ROSAMONTE PLUS SUAVE 10x1 KG,10,SUSPEN,118,1,19,2,181,*YBA ROSAMONTE PLUS SUAVE 10x1 KG,10.0,SUSPEN,118.0,1.0,23,2,181
33905,9984,*YBA ROSAMONTE PLUS SUAVE 10x500 GR,10,SUSPEN,118,1,19,2,181,*YBA ROSAMONTE PLUS SUAVE 10x500 GR,10.0,SUSPEN,118.0,1.0,23,2,181
33910,9989,GLADE SENSAT BAO LAVANDA X 6 U,6,INACTI,115,5,9,10,179,GLADE SENSAT BAO LAVANDA X 6 U,6.0,INACTI,115.0,5.0,64,10,179
33911,9990,COMBO RAID AERO+TABLETAS+APARATO x 1 U,1,SUSPEN,115,5,50,3,179,COMBO RAID AERO+TABLETAS+APARATO x 1 U,1.0,SUSPEN,115.0,5.0,66,3,179


In [30]:
try:
    with db.cursor() as cursor:
        for index, row in diferencias.iterrows():
            update_query = """
                UPDATE maestro_articulos
                SET CodFamilia = %s
                WHERE Codigo = %s;
            """
            cursor.execute(update_query, (row['codfamilia'], row['Codigo']))
    
    db.commit()
finally:
    db.close()

In [5]:
df_maestro.drop_duplicates(inplace= True)

Cargamos los datos en la BD

In [5]:
for _, fila in df_linea.iterrows():
    query = "INSERT INTO linea (CodLinea, Descr) VALUES (%s, %s)"
    values = (fila['codlinea'], fila['desclinea'])
    cursor.execute(query, values)

db.commit()

In [6]:
for _, fila in df_sector.iterrows():
    query = "INSERT INTO sector (CodSector, Descr) VALUES (%s, %s)"
    values = (fila['codsector'], fila['descsector'])
    cursor.execute(query, values)

db.commit()

In [5]:
for _, fila in df_proveedor.iterrows():
    query = "INSERT INTO proveedor (CodProveedor, Descr) VALUES (%s, %s)"
    values = (fila['codproveedor'], fila['proveedor'])
    cursor.execute(query, values)

db.commit()

In [6]:
for _, fila in df_familia.iterrows():
    query = "INSERT INTO familia (CodFamilia, Descr) VALUES (%s, %s)"
    values = (fila['codfamilia'], fila['descfamilia'])
    cursor.execute(query, values)

db.commit()

In [7]:
for _, fila in df_subfamilia.iterrows():
    query = "INSERT INTO subfamilia (CodSubFamilia, Descr) VALUES (%s, %s)"
    values = (fila['codsubfamilia'], fila['descsubfamilia'])
    cursor.execute(query, values)

db.commit()

In [6]:
for _, fila in df_maestro.iterrows():
    query = "INSERT INTO maestro_articulos (Codigo, Descripcion, UxB, Estado, CodLinea, CodSector, CodFamilia, CodSubFamilia, CodProveedor ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    values = (fila['codart'], fila['descart'], fila['unidxbult'], fila['estado'], fila['codlinea'], fila['codsector'], fila['codfamilia'], fila['codsubfamilia'], fila['codproveedor'])
    cursor.execute(query, values)

db.commit()

In [16]:
df = pd.read_excel('datos_iniciales_roturas.xlsx')

In [17]:
from datetime import datetime

# Supongamos que este es tu DataFrame
# df = pd.DataFrame(...)

# Asegurándote de que la columna 'FECHA' es un objeto datetime
df['FECHA'] = pd.to_datetime(df['FECHA'])

# Ahora, formateando la columna 'FECHA' al formato que MySQL espera
df['FECHA'] = df['FECHA'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))


In [18]:
# Convertir la columna 'CÓDIGO' a tipo string
df['CÓDIGO'] = df['CÓDIGO'].astype(str)

# Eliminar el '.0' del final de los códigos
df['CÓDIGO'] = df['CÓDIGO'].apply(lambda x: x[:-2] if x.endswith('.0') else x)

In [19]:
# Abre un archivo para registrar los errores
with open("errores_log.txt", "w") as error_log:

    for index, fila in df.iterrows():
        try:
            # Preparar tu consulta SQL y los valores
            query = "INSERT INTO Roturas (Fecha, Codigo, Cantidad, Costo, Motivo) VALUES (%s, %s, %s, %s, %s)"
            values = (fila['FECHA'], fila['CÓDIGO'], fila['CANTIDAD'], fila['COSTO'], fila['TIPO'])

            # Ejecutar la consulta
            cursor.execute(query, values)
            db.commit()

        except mysql.connector.IntegrityError as e:
            print(f"Error al insertar la fila {index}: {e}")
            
            # Escribir el error y el código problemático en el archivo de log
            error_log.write(f"Fila {index}, Código: {fila['CÓDIGO']}, Error: {e}\n")
            
            # Continuar con la siguiente fila
            continue


Error al insertar la fila 1: 1062 (23000): Duplicate entry '10881-2023-12-13-ROTURA' for key 'Roturas.PRIMARY'
Error al insertar la fila 2: 1062 (23000): Duplicate entry '11194-2023-12-13-ROTURA' for key 'Roturas.PRIMARY'
Error al insertar la fila 3: 1062 (23000): Duplicate entry '11183-2023-12-13-ROTURA' for key 'Roturas.PRIMARY'
Error al insertar la fila 4: 1062 (23000): Duplicate entry '11138-2023-12-13-ROTURA' for key 'Roturas.PRIMARY'
Error al insertar la fila 5: 1062 (23000): Duplicate entry '10919-2023-12-13-VENCIMIENTO' for key 'Roturas.PRIMARY'
Error al insertar la fila 6: 1062 (23000): Duplicate entry '10904-2023-12-13-VENCIMIENTO' for key 'Roturas.PRIMARY'
Error al insertar la fila 7: 1062 (23000): Duplicate entry '10904-2023-12-13-ROTURA' for key 'Roturas.PRIMARY'
Error al insertar la fila 8: 1062 (23000): Duplicate entry '10882-2023-12-13-VENCIMIENTO' for key 'Roturas.PRIMARY'
Error al insertar la fila 9: 1062 (23000): Duplicate entry '10879-2023-12-13-VENCIMIENTO' for key

In [4]:
df = pd.read_excel('codigos_no_encontrados_descripciones.xlsx')

In [5]:
df.dropna(inplace= True)

In [6]:
for index, fila in df.iterrows():
    try:
        # Preparar tu consulta SQL y los valores
        query = "INSERT INTO maestro_articulos (Codigo, Descripcion, UxB, Estado, CodLinea, CodSector, CodFamilia, CodSubFamilia, CodProveedor ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (fila['CÓDIGO'], fila['DESCRIPCIÓN'], fila['UxB'], fila['Estado'], fila['CodLinea'], fila['CodSector'], fila['CodFamilia'], fila['CodSubFamilia'], fila['CodProveedor'])

        # Ejecutar la consulta
        cursor.execute(query, values)
        db.commit()

    except mysql.connector.IntegrityError as e:
        print(f"Error al insertar la fila {index}: {e}")
        
        # Continuar con la siguiente fila
        continue

Error al insertar la fila 0: 1062 (23000): Duplicate entry '26847' for key 'maestro_articulos.PRIMARY'
Error al insertar la fila 2: 1062 (23000): Duplicate entry '26518' for key 'maestro_articulos.PRIMARY'
Error al insertar la fila 3: 1062 (23000): Duplicate entry '26512' for key 'maestro_articulos.PRIMARY'
Error al insertar la fila 5: 1062 (23000): Duplicate entry '26044' for key 'maestro_articulos.PRIMARY'
Error al insertar la fila 6: 1062 (23000): Duplicate entry '26003' for key 'maestro_articulos.PRIMARY'
Error al insertar la fila 7: 1062 (23000): Duplicate entry '25865' for key 'maestro_articulos.PRIMARY'
Error al insertar la fila 8: 1062 (23000): Duplicate entry '25841' for key 'maestro_articulos.PRIMARY'
Error al insertar la fila 9: 1062 (23000): Duplicate entry '26497' for key 'maestro_articulos.PRIMARY'
Error al insertar la fila 10: 1062 (23000): Duplicate entry '26414' for key 'maestro_articulos.PRIMARY'
Error al insertar la fila 11: 1062 (23000): Duplicate entry '26001' for 